# Discrete Dynamic Programing

After the previous Value Function Iteration (VFI) notebooks it's time for talking discrete dynamic programming at large.

The setup here is different from the previous problem (see VFI 1.5), where we took a [monte carlo integration](https://en.wikipedia.org/wiki/Monte_Carlo_integration) approach to deal with uncertainty - we took a sample of random numbers (large enough) from the distribution of shocks and then took the mean. That way we approximated the value of the expectation operator (up to an error).

There is another way of taking expectations 

This notebook is to be used after reading the first part (up until *Example: A Growth Model*) of the [quantecon lecture](https://lectures.quantecon.org/py/discrete_dp.html) on dynamic programming. Read it in your own (if you are not seeing this in class) first, making sure you follow fairly well what it says. The more difficult bits (value function interation) should be solid from our previous notebooks.

You'll need numba, so install it now before proceeding. The folowing cell should execute without problems:

In [ ]:
from __future__ import division  # Not needed for Python 3.x
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import quantecon as qe
from numba import jit

sns.set_style('whitegrid')
%matplotlib inline

## 1.1 A cookie problem

The aim of this bit is to provide you with a reference guide so you cna implement your own versions in the future.

The problem follows [the growth model example](https://lectures.quantecon.org/py/discrete_dp.html#example-a-growth-model) of quantecon.

Suppose we live in a world where cookies are the only commodity that matters.

Every night, the cookie fairies bring a random ammount of cookies to your door. The amount of cookies ($y$) received is distributed uniformly from 0 to B:

$$ y \sim U[0,B] $$

So chances of getting 5 cookies are the same as taking 0.

You are endowed with a cookie jar that can hold at maximum M cookies.

You wake up in the morning, take your "income" cookies and put them together with your exisiting savings in the jar.

Your goal then is to decide how many cookies out of your income ($y$) you want to save ($a$) and how many you want to consume ($c$) today (so that $y = c+a$). Consumed cookies give you a unitily of $u(c)$ and you discount the future by $\beta$ each period.

Let's make utility exponential for this case so that $u(c)=c^{\alpha}$.

> **Task 1:** Following the [quantecon](https://lectures.quantecon.org/py/discrete_dp.html#example-a-growth-model) exposition, explain with your own words what is the state variable, state space, action space, reward function and transition probabilities. No reading the math, but explaining with words.

> Then use them to add comments to the setting cell below explaining what each line represents.

In [ ]:
M=10             # ???
B=5              # ???
n = B + M + 1    # ???
m = M + 1        # ???
alpha=0.5        # ???
beta=0.9         # ???

We are going to use discrete markov chains to incorporate the random component of $y$ into our decisions. For that we need to code the matrix algebra version of this problem:

$$ v(s) = \max_a r(s,a) + \beta \sum_{s' \in S}Q(s,a,s') v(s')$$

- $Q$ is going to be a **tri**dimensional matrix that relates states today, actions today and states tomorrow.

- $r(s,a)$ is going to be a matrix of utilitiesvalues, for each state-value pair $(s,a)$: $u(s-a)$. We are going to "cheat" by giving a utility of $-\infty$ to the unfeasible values (say you have 0 cookie in the jar and you receive 1: You can only save either 1 or 0. If you choose to save more then you get minus infinity utility).

That's why we are going to code them in the following way (make sure you understand it):

In [ ]:
R = np.empty((n, m))
Q = np.zeros((n, m, n))

Now it is time to fill up the matrices.

> **Task 2:** Complete the code below. You can do this in easy mode (go back to quantecon and copy the bits you need) or in hard mode: pen and paper if needed and coding it by hand.

In [ ]:
# Utility function


In [ ]:
# Reward matrix


In [ ]:
# Markow transition matrix


We can use the cells below to print the matrices and check we did the right thing.

Or we can plot them too:

In [ ]:
plt.plot(R.T)
plt.xlabel('action $a$')
plt.ylabel('reward $u(a)$')
plt.title('Each line represents a state $s$')
plt.show()

## 1.2 Solving for cookies: home made solution

We could go ahead and use the quantecon package for this problem, but I think it would be more useful if we know what are we doing and code the algorithms ourselves. If you don't feel adventurous you can skip to the next section.

First let's solve it by VFI. We will have:

$$ V_{t+1} = \max_a R + \beta Q V_{t}$$

Our algorithm will be as follows:

- We give a initial guess $V_0$ for $V_{t}$. We will have a resulting $n$ x $m$ matrix (call it *RQV*)
- For each row of *RQV*, choose the column that gives the maximum value. This is the $\max_a$ step.
- Given this choice, calculate the vector $V_{t+1}$ (put all of the maximum values together in one vector)
- Calculate the distance between $V_{t+1}$ and $V_{t}$
- If this distance is greater than some tolerance, set our guess vector $V_{t} = V_{t+1}$ and repeat.

Let's do it slowly so we get a grasp of what's going on.

**1** We set our initial guess, a vector of size $n$

In [ ]:
v0 = np.ones(n)
v0

**2** We compute the implied RQV matrix.

Notive the use of ``np.dot``: if you use ``*`` you get *pointwise* multiplication, we don't want that for $Q$ and $V$, but we want that for $\beta$.

In [ ]:
RQV = R + beta*np.dot(Q,v0)
print RQV 

**3** We find the column (action) that gives the maximun value of each row (state).

For this we use numpy function maximum, as it allows us to chose the direction (1 for column, 0 for row).

We can also record our choices into a vector (of which lenght?). Call it $p$ for policy.

In [ ]:
np.max(RQV,1)

In [ ]:
np.argmax(RQV,1)

In [ ]:
v1  = np.max(RQV,1)
p1 = np.argmax(RQV,1)

**4** In equilibirum, ``v0=v1``. Let's check the maximum difference between both after one iteration.

In [ ]:
np.max(np.abs(v1-v0))

Now that we understand the steps, it's time to implement the algorithm.

> **Task 3:** Code the Value Function Iteration algorithm, following the steps above.

In [ ]:
# Your code here


print "Done! Distance: {}  Iterations: {}".format(distance,it)

If the code worked out correctly, the following cell should give you the answer

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
plt.plot(range(n),v1)
plt.xlim(0,15)
plt.xticks(range(0,16))
plt.ylabel('$v^*(s)$', fontsize=16)
plt.xlabel('$s$', fontsize=16)
plt.title('Our value function', fontsize=16)

plt.subplot(122)
plt.plot(range(n),p1, label='cookies saved')
plt.plot(range(n),np.arange(n)-p1, label='cookies consumed')
plt.xlim(0,15)
plt.xticks(range(0,16))
plt.title('Our (discrete) policy function', fontsize=16)
plt.ylabel('$a^*(s)$, $c*(a(s))$', fontsize=16)
plt.xlabel('$s$', fontsize=16)
plt.legend(loc='best')
plt.show()

plt.show()

## 1.3 Solving for cookies: Quantecon solution

That was fast! But did we got the right answer? Let's check using quantecon's tools.

The following lines create an *instance* (a variable containing a problem) of the ``discreteDP`` class, then solve it.

In [ ]:
ddp = qe.markov.DiscreteDP(R, Q, beta)
results = ddp.solve(method='value_iteration')

The variable ``results`` is a dictionary  that stores the output from the algorithm. Call ``.keys()`` to see what it recorded.

In [ ]:
results.keys()

> **Task 4:** Select the right result to store the policy function in ``p_qe`` and the value function ``v_qe``

In [ ]:
v_qe = results.v
p_qe = results.sigma

Do we got the right answer?

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(121)
plt.plot(range(n),v1, label='homemade')
plt.plot(range(n),v_qe, label='quant-econ')
plt.xlim(0,15)
plt.xticks(range(0,16))
plt.ylabel('$v^*(s)$', fontsize=16)
plt.xlabel('$s$', fontsize=16)
plt.legend(loc='best', frameon=True, fontsize=14)
plt.title('Our value function', fontsize=16)

plt.subplot(122)
plt.plot(range(n),p1, label='homemade')
plt.plot(range(n),p_qe, label='quant-econ')
plt.xlim(0,15)
plt.xticks(range(0,16))
plt.title('Our (discrete) policy function', fontsize=16)
plt.ylabel('$a^*(s)$', fontsize=16)
plt.xlabel('$s$', fontsize=16)
plt.legend(loc='best', frameon=True, fontsize=14)

plt.show()

Hurray! But mind you, the routines of the quantecon function are more sophisticated that ours, so they solve the problem faster. Compare the number of iterations it took to get to the answer:

In [ ]:
print "Our number of iterations: {}".format(it)
print "QuantEcon number of iterations: {}".format(results.num_iter)

## 2. Back to our growth model

Can we use the tools we learnt to get to the same answer we got last week?

This section goes through the answer to the first exercise of the quantecon lecture. But if you are up for a real challenge, try to do it yourself! Note that it is quite hard.

The exercise constitutes and example of how to present a problem in a way you can use the quantecon routine to solve it. Which is, incidentally, what you will end up doing in coding: finding the right tools for your problem and adapting your problem  so you can use them effectively.

The first step is to recode the main parameters (including the capital grid) and the analytical answer. There is no productivity shock.

In [ ]:
# Defining Parameters
alpha = 0.65
f = lambda k: k**alpha
u = np.log
beta = 0.95

# Defining grid
grid_max = 2
grid_size = 200
grid = np.linspace(1e-6, grid_max, grid_size)

# Exact solution (for comparison)
ab = alpha * beta
c1 = (np.log(1 - ab) + np.log(ab) * ab / (1 - ab)) / (1 - beta)
c2 = alpha / (1 - ab)

def v_star(k):
    return c1 + c2 * np.log(k)

true_c = (1 - alpha * beta) * grid**alpha

Then let's define our feasible set $A(s)$. This is going to ge all of the combinations of state (capital) and action (consumption) that yield non-negative state tomorrow (capital tomorrow).

We use the ``np.where`` function to locate these values.  ``np.where`` returns two vectors: one with the row indexes and one with the column indexes. For example if our set was:

```
[[0,0,-9],
[0,-1,26]]
```

Then ``np.where`` would return: ``[0,0,1,1],[0,1,0,2]``. The coordinates of all of the points that are greater than 0.

In [ ]:
# Consumption matrix, with nonpositive consumption included
As = f(grid).reshape(grid_size, 1) - grid.reshape(1, grid_size)
# size 150x1, 1x150

# State-action indices
s_indices, a_indices = np.where(As > 0)

# Number of state-action pairs
L = len(s_indices)

Of course ``s_indices`` and ``a_indices`` conatin the same number of elements ``L``. You can check what the variables contain using the print cells below:

In [ ]:
print As

In [ ]:
print s_indices, a_indices

Makes sense, right?

So what are going to be our R and Q matrices? The ones corresponding to the feasible state-action pairs. For that reason, the reward matrix $R$ is going to be a vector this time (each element corresponding to a state-action) of length $1$x$L$.

Now for $Q$: there is no uncertainty, so $Q$ will map action pairs $As$ to the corresponding level of capital tomorrow. So each row will map each action pair to the state it came from $A(s) \rightarrow s$. This is because as in the no uncertainty case:

$$ V(k) = \max_{k'} {u(k') + \beta V(k')}$$

So it does make sense that the $k'$ in $V(k')$ should be the same as in $V(k)$ with probability 1.

In [ ]:
R = u(As[s_indices, a_indices])
Q = np.zeros((L, grid_size))    # creates a 'sparse' matrix, with all zeros

Q[np.arange(L), a_indices] = 1          

Don't worry if it is confusing. This notation is used by the ``DiscreteDP`` class to solve the problem:

In [ ]:
ddp2 = qe.markov.DiscreteDP(R, Q, beta, s_indices, a_indices)
results2 = ddp2.solve(method='policy_iteration')
v_qe2 = results2.v               # Value function
p_qe2 = results2.sigma           # Optimal policy (savings)
c_qe2 = f(grid) - grid[p_qe2]    # Optimal consumption

And as you can see, it solves the problem correctly:

In [ ]:
plt.figure(figsize=(14,6))
plt.subplot(121)
plt.plot(grid,v_qe2, label='Discrete $c^*(a_i)$')
plt.plot(grid,v_star(grid), label='True $c^*(a_i)$')
plt.ylim(-42, -32)
plt.ylabel('$v^*(a_i)$', fontsize=12)
plt.xlabel('$a_i$', fontsize=12)
plt.legend(loc='best',fontsize=12, frameon=True)
plt.title('Comparing the value function', fontsize=14)

plt.subplot(122)
plt.plot(grid,v_qe2, label='Discrete $c^*(a_i)$')
plt.plot(grid,v_star(grid), label='True $c^*(a_i)$')
plt.ylim(-35.2, -34.4)
plt.xlim(0.8,1.2)
plt.xlabel('$a_i$', fontsize=12)
plt.title('Comparing the value function', fontsize=14)
plt.title('Zoom around 1', fontsize=14)

plt.show()

In [ ]:
# Zoom in
plt.figure(figsize=(10,6))
plt.plot(grid,c_qe2, label='Discrete $c^*(a_i)$')
plt.plot(grid,true_c, color='black', alpha=0.5, label='True $c^*(a_i)$')
plt.title('Policy function (zoom around 1)', fontsize=18)
plt.xlabel('$a_i$', fontsize=16)
plt.ylabel('$c^*(a_i)$', fontsize=16)
plt.xlim(0.8,1.2)
plt.ylim(0.3,0.45)
plt.legend(loc='best',fontsize=16, frameon=True)
plt.show()

## 3. Take aways and results

As you can see, it can be tricky to work with markov chain transitions, but the speed and accuracy of results is worth it.

From here, I propose you pick up on of the next exercises to do for next session:

1. Implement the policy function iteration to the cookie problem. I haven't had time to talk about it this time, but it is a great opportunity to do so in the next one.

2. Modify the code above to include the innovation as we saw it in VFI 1.5. This is quite a challenge, because it requires first to transform the lognromal shock into a markov transition matrix, and then specify the Q and R matrices correctly. It is great for understanding though, and certainly useful for the future.

3. More fun: Read the [McCall model lecture](https://lectures.quantecon.org/py/mccall_model.html) and add to it assets - that is, a grid for assets and a consumption-saving decision. Compare the wage reservation you get to the 'hand-to-mouth' scenario.

4. Play around risk aversion using different utility functions. Do you get the result you expected?

And after that we will conclude our introduction to dynamic programming.